In [ ]:
!pip install ortools

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.7/27.7 MB 56.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.8/135.8 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 321.1/321.1 kB 20.7 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 5.29.5
    Uninstalling protobuf-5.29.5:
      Successfully uninstalled protobuf-5.29.5
  Attempting uninstall: absl-py
    Found existing installation: absl-py 1.4.0
    Uninstalling absl-py-1.4.0:
      Successfully uninstalled absl-py-1.4.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
grpcio-status 1.71.2 requires protobuf<6.0dev,>=5.26.1, but you have protobuf 6.31.1 which is incompatible.
tensorflow 2.19.0 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<6.0.0dev,>=3.20.3, but you have protobuf 6.31.1 which is incompatibl

In [ ]:
from ortools.linear_solver import pywraplp

In [ ]:
# declara o solver: se PLI, usa-se SCIP; se PL, usa-se GLOP.
solver = pywraplp.Solver.CreateSolver('SCIP')

# declara um número muito grande caso seja necessário usar no modelo
infinity = solver.infinity()

In [ ]:
# declara as variáveis do modelo: IntVar para inteira; NumVar para fracionárias; BoolVar para binárias.
x1 = solver.IntVar(0, infinity, 'x1')     # quantidade de paraquedas
x2 = solver.IntVar(0, infinity, 'x2')     # quantidade de asa-delta

In [ ]:
# declara a função objetivo
objetivo = solver.Objective()
objetivo.SetCoefficient(x1, 40)
objetivo.SetCoefficient(x2, 10)
objetivo.SetMaximization()                # SetMinimization para minimização

In [ ]:
# declara uma restrição
restricao_1 = solver.RowConstraint(-infinity, 100, 'linha_1')       # colocar um nome para a restrição é opcional
restricao_1.SetCoefficient(x1, 2)
restricao_1.SetCoefficient(x2, 3)

In [ ]:
# declara uma restrição
restricao_2 = solver.RowConstraint(-infinity, 42, 'linha_2')        # colocar um nome para a restrição é opcional
restricao_2.SetCoefficient(x1, 2)
restricao_2.SetCoefficient(x2, 1)

In [ ]:
# resolve o modelo
status = solver.Solve()

In [ ]:
# verifica se a solução é ótima e, caso seja, exibe ela
if status == pywraplp.Solver.OPTIMAL:
  print('Lucro:', objetivo.Value())
  print(x1.name(), x1.solution_value())
  print(x2.name(), x2.solution_value())
else:
  print('Modelo sem solução ótima.')

Lucro: 840.0
x1 21.0
x2 0.0


In [ ]:
# exibe o modelo
print(solver.ExportModelAsLpFormat(False))

\ Generated by MPModelProtoExporter
\   Name             : 
\   Format           : Free
\   Constraints      : 2
\   Variables        : 2
\     Binary         : 0
\     Integer        : 2
\     Continuous     : 0
Maximize
 Obj: +40 x1 +10 x2 
Subject to
 linha_1: +2 x1 +3 x2  <= 100
 linha_2: +2 x1 +1 x2  <= 42
Bounds
 0 <= x1 <= inf
 0 <= x2 <= inf
Generals
 x1
 x2
End



In [ ]:
# declara o solver: neste caso, usaremos SCIP para variáveis inteiras
solver = pywraplp.Solver.CreateSolver('SCIP')

# declara um número muito grande caso seja necessário usar no modelo
infinity = solver.infinity()

In [ ]:
# declara as variáveis do modelo:
# x_i representa o número de motoristas que iniciam seu plantão no período i, para i = 1 a 6.
x = [solver.IntVar(0, infinity, f'x{i+1}') for i in range(6)]

In [ ]:
# declara a função objetivo: minimizar o número total de motoristas
objetivo = solver.Objective()
for i in range(6):
  objetivo.SetCoefficient(x[i], 1)
objetivo.SetMinimization()

In [ ]:
# declara as restrições: número mínimo de motoristas em cada período
# Cada motorista trabalha por 8 horas, cobrindo dois períodos consecutivos.
# Período 1 (1-5): motoristas que começam no período 1 ou período 6 (que termina no período 1)
restricao_1 = solver.RowConstraint(15, infinity, 'periodo_1')
restricao_1.SetCoefficient(x[0], 1)
restricao_1.SetCoefficient(x[5], 1) # Motoristas que começam no período 6 trabalham até o fim do período 1

# Período 2 (5-9): motoristas que começam no período 1 ou período 2
restricao_2 = solver.RowConstraint(30, infinity, 'periodo_2')
restricao_2.SetCoefficient(x[0], 1)
restricao_2.SetCoefficient(x[1], 1)

# Período 3 (9-13): motoristas que começam no período 2 ou período 3
restricao_3 = solver.RowConstraint(26, infinity, 'periodo_3')
restricao_3.SetCoefficient(x[1], 1)
restricao_3.SetCoefficient(x[2], 1)

# Período 4 (13-17): motoristas que começam no período 3 ou período 4
restricao_4 = solver.RowConstraint(32, infinity, 'periodo_4')
restricao_4.SetCoefficient(x[2], 1)
restricao_4.SetCoefficient(x[3], 1)

# Período 5 (17-21): motoristas que começam no período 4 ou período 5
restricao_5 = solver.RowConstraint(30, infinity, 'periodo_5')
restricao_5.SetCoefficient(x[3], 1)
restricao_5.SetCoefficient(x[4], 1)

# Período 6 (21-1): motoristas que começam no período 5 ou período 6
restricao_6 = solver.RowConstraint(19, infinity, 'periodo_6')
restricao_6.SetCoefficient(x[4], 1)
restricao_6.SetCoefficient(x[5], 1)

In [ ]:
# resolve o modelo
status = solver.Solve()

In [ ]:
# verifica se a solução é ótima e, caso seja, exibe ela
if status == pywraplp.Solver.OPTIMAL:
  print('Número total de motoristas:', int(objetivo.Value()))
  for i in range(6):
    print(x[i].name(), int(x[i].solution_value()))
else:
  print('Modelo sem solução ótima.')

Número total de motoristas: 81
x1 0
x2 30
x3 0
x4 32
x5 4
x6 15


In [ ]:
# exibe o modelo
print(solver.ExportModelAsLpFormat(False))

\ Generated by MPModelProtoExporter
\   Name             : 
\   Format           : Free
\   Constraints      : 6
\   Variables        : 6
\     Binary         : 0
\     Integer        : 6
\     Continuous     : 0
Minimize
 Obj: +1 x1 +1 x2 +1 x3 +1 x4 +1 x5 +1 x6 
Subject to
 periodo_1: +1 x1 +1 x6  >= 15
 periodo_2: +1 x1 +1 x2  >= 30
 periodo_3: +1 x2 +1 x3  >= 26
 periodo_4: +1 x3 +1 x4  >= 32
 periodo_5: +1 x4 +1 x5  >= 30
 periodo_6: +1 x5 +1 x6  >= 19
Bounds
 0 <= x1 <= inf
 0 <= x2 <= inf
 0 <= x3 <= inf
 0 <= x4 <= inf
 0 <= x5 <= inf
 0 <= x6 <= inf
Generals
 x1
 x2
 x3
 x4
 x5
 x6
End

